In [1]:
import pandas as pd
import recsys as rs
import ItemBasedRecommender as IBR
import json
import time

In [2]:
data = pd.read_csv('Data/train_final.csv','\t')
tracks = pd.read_csv('Data/tracks_final.csv','\t')

In [3]:
train, test, tgt_tracks, tgt_playlists = rs.train_test_split_interface(data, 10, 20, 5, 2517)

In [4]:
fit_dict_ibr = {'idf' : True,
                'measure' : 'dot',
                'shrinkage' : 10,
                'n_el_sim' : 65}

In [5]:
fitstart = time.time()
ibr = IBR.ItemBasedRecommender(**fit_dict_ibr)
ibr.fit(tracks,train,tgt_tracks, multiprocessing=False)
fitend = time.time()

Calculated Indices
Model URM built
Model URM regularized with IDF!


100%|██████████| 100000/100000 [42:30<00:00, 39.21it/s]


In [6]:
recstart = time.time()
recs = ibr.recommend(tgt_playlists, train, sim_check=False, secondary_sorting=False, multiprocessing=False)
recend = time.time()

  0%|          | 0/9129 [00:00<?, ?it/s]

URM built


100%|██████████| 9129/9129 [43:11<00:00,  3.52it/s]


In [7]:
print('Fit completed in '+str(float(fitend-fitstart))+' seconds.')
print('Recommendetion completed in '+str(float(recend-recstart))+' seconds.')

Fit completed in 2551.27481007576 seconds.
Recommendetion completed in 2591.5404880046844 seconds.


In [8]:
map_eval = rs.evaluate(recs, test, 'MAP')
print('MAP score: '+str(map_eval))

MAP score: 0.03879870011319247


In [9]:
run_data = {'recommender' : ibr.__class__.__name__,
            'fit_parameters' : fit_dict_ibr,
            'evaluation' : map_eval}

In [10]:
with open('runs_data.json', 'a') as fp:
    json.dump(run_data, fp, indent=2)
    fp.write('\n')

In [11]:
import notipy
notipy.notify('MAP score: '+str(map_eval))